## Odds Joining (OLD: 2018 Only)

The purpose of this notebook is to determine how to join odds data, from [here](http://www.tennis-data.co.uk/alldata.php), with the parsed data.  This is difficult because player's names are represented differently, the parsed data doesn't have match dates, tournaments are represented differently, and so on.  There are various manual corrections we will need to make to create our basis for joining the data here.

Here, we join by player name, tournament, and score.  This produces a dataframe called merged_2018.csv

#### Joining Odds Data to My Data

In [1]:
import pandas as pd
from tennis_new.fetch.get_joined import read_joined

odds_df = pd.read_csv("/Users/siddhantjagadish/Downloads/2018.csv")
jd = read_joined()

/Users/siddhantjagadish/Documents/DataProjects/tennis_new/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [2]:
jd.shape, odds_df.shape

((373236, 28), (2637, 40))

#### Explore Odds Data

In [3]:
odds_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,EXW,EXL,LBW,LBL,PSW,PSL,MaxW,MaxL,AvgW,AvgL
0,1,Brisbane,Brisbane International,12/31/17,ATP250,Outdoor,Hard,1st Round,3,Dolgopolov O.,...,2.10,1.65,2.20,1.67,2.24,1.71,2.29,1.77,2.17,1.68
1,1,Brisbane,Brisbane International,12/31/17,ATP250,Outdoor,Hard,1st Round,3,De Minaur A.,...,2.80,1.38,2.75,1.44,3.14,1.41,3.25,1.44,2.96,1.39
2,1,Brisbane,Brisbane International,1/1/18,ATP250,Outdoor,Hard,1st Round,3,Harrison R.,...,1.62,2.15,1.67,2.20,1.66,2.35,1.75,2.35,1.67,2.17
3,1,Brisbane,Brisbane International,1/1/18,ATP250,Outdoor,Hard,1st Round,3,Ebden M.,...,2.35,1.53,2.37,1.57,2.38,1.64,2.52,1.65,2.37,1.58
4,1,Brisbane,Brisbane International,1/1/18,ATP250,Outdoor,Hard,1st Round,3,Zverev M.,...,1.47,2.50,1.57,2.37,1.44,3.00,1.57,3.00,1.44,2.73


#### Name Processing

Let's process the player's names so we can join on them

In [165]:
JD_NAME_CORRECTIONS = {
    'MARTIN DEL POTRO': 'DEL POTRO',
    'IGNACIO LONDERO': 'LONDERO',
    'AL-MUTAWA': 'AL MUTAWA',
    'FERREIRA SILVA': "SILVA",
    "ELAHI GALAN": "GALAN"
}

In [166]:
# Preprocessing for upper case
jd['winner_name'] = jd['winner_name'].map(lambda x: x.upper())
jd['loser_name'] = jd['loser_name'].map(lambda x: x.upper())
odds_df['Winner'] = odds_df['Winner'].map(lambda x: x.upper())
odds_df['Loser'] = odds_df['Loser'].map(lambda x: x.upper())

In [167]:
def last_name_jd(n):
    return ' '.join(n.upper().split(' ')[1:])

def last_name_odds(n):
    return ' '.join(n.upper().split(' ')[: -1])

jd['winner_last_name'] = jd['winner_name'].map(last_name_jd)
jd['loser_last_name'] = jd['loser_name'].map(last_name_jd)
jd['winner_last_name'] = jd['winner_last_name'].map(lambda x: JD_NAME_CORRECTIONS.get(x, x))
jd['loser_last_name'] = jd['loser_last_name'].map(lambda x: JD_NAME_CORRECTIONS.get(x, x))

In [168]:
odds_df['winner_last_name'] = odds_df['Winner'].map(last_name_odds)
odds_df['loser_last_name'] = odds_df['Loser'].map(last_name_odds)

In [169]:
top_missing_winners = odds_df['Winner'][~odds_df['winner_last_name'].isin(jd['winner_last_name'])].value_counts()
top_missing_winners                              

Series([], Name: Winner, dtype: int64)

In [170]:
top_missing_losers = odds_df['Loser'][~odds_df['loser_last_name'].isin(jd['loser_last_name'])].value_counts()
top_missing_losers                              

Series([], Name: Loser, dtype: int64)

In [171]:
PROBLEM_NAME = 'AL MUTAWA J.'
odds_df[odds_df['Loser'] == PROBLEM_NAME]

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,MaxW,MaxL,AvgW,AvgL,winner_last_name,loser_last_name,odds_match_id
39,2,Doha,QATAR EXXON MOBIL OPEN,1/2/18,ATP250,Outdoor,Hard,1st Round,3,TRAVAGLIA S.,AL MUTAWA J.,134,NaN,414,NaN,6.0,1.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.005,34.0,NaN,NaN,1.01,26.0,1.01,25.62,1.02,34.0,1.01,18.89,TRAVAGLIA,AL MUTAWA,39


In [172]:
odds_df[['Loser', 'loser_last_name']][
    odds_df['Loser'].isin(top_missing_losers.index)
].drop_duplicates('Loser')

,Loser,loser_last_name


#### Tournament Mapping

The tournament names are presented differently as well.  We will have to map tournament names to each other to provide the right mappings...

In [12]:
# Preprocessing, don't wanna deal with case issues

jd['tourney_title'] = jd['tourney_title'].map(lambda x: x.upper())
odds_df['Tournament'] = odds_df['Tournament'].map(lambda x: x.upper().strip(' '))

In [13]:
odds_df['Tournament'][~odds_df['Tournament'].isin(jd['tourney_title'])].value_counts().head()

FRENCH OPEN            127
SONY ERICSSON OPEN      95
ROGERS MASTERS          55
MONTE CARLO MASTERS     55
SHANGHAI MASTERS        55
Name: Tournament, dtype: int64

In [14]:
pd.set_option('display.max_rows', 100, 'display.max_columns', None) # more options can be specified also
wubba = odds_df.groupby(['Tournament', 'Location']).apply(
    lambda x: pd.Series({
        'n_matches': x.shape[0],
        'min_date': x['Date'].min()
    })
).sort_values('n_matches', ascending=False)
wubba

,,n_matches,min_date
Tournament,Location,,
WIMBLEDON,London,127,7/10/18
US OPEN,New York,127,8/27/18
AUSTRALIAN OPEN,Melbourne,127,1/15/18
FRENCH OPEN,Paris,127,5/27/18
SONY ERICSSON OPEN,Miami,95,3/21/18
BNP PARIBAS OPEN,Indian Wells,95,3/10/18
INTERNAZIONALI BNL D'ITALIA,Rome,55,5/13/18
WESTERN & SOUTHERN FINANCIAL GROUP MASTERS,Cincinnati,55,8/12/18
SHANGHAI MASTERS,Shanghai,55,10/10/18


In [15]:
def tourney_stats(s):
    tourney_titles = jd[
        jd['tourney_title'].map(lambda x: s in x) &
        (jd['tour_type'] == 'atp')
    ]['tourney_title'].unique()
    rel = jd[
        jd['tourney_title'].isin(tourney_titles) &
        (jd['tour_type'] == 'atp')
    ]
    print(rel['tourney_title'].value_counts())
    print(rel.groupby('tourney_title').apply(lambda x: (x['tourney_dates'].min(), x['tourney_dates'].max())))
    
tourney_stats("BNP")

BNP PARIBAS OPEN    131
Name: tourney_title, dtype: int64
tourney_title
BNP PARIBAS OPEN    (2019.03.07, 2019.03.07)
dtype: object


In [138]:
# To help with joining odds data...
TOURNAMENT_MAPPING = {
    'BRISBANE INTERNATIONAL': ['BRISBANE', 'BRISBANE INTERNATIONAL'],
    'US OPEN': 'US OPEN',
    'WIMBLEDON': 'WIMBLEDON',
    'FRENCH OPEN': 'ROLAND GARROS',
    'AUSTRALIAN OPEN': ['AUSTRALIAN OPEN',  'AUSTRALIAN OPEN-2'],
    'SONY ERICSSON OPEN': [
        'ATP MASTERS 1000 MIAMI',
        'MIAMI OPEN PRESENTED BY ITAU',
        'MIAMI'
    ],
    "BNP PARIBAS OPEN": ['BNP PARIBAS OPEN', 'ATP MASTERS 1000 INDIAN WELLS', 'INDIAN WELLS'],
    "BNP PARIBAS MASTERS": ['BNP PARIBAS MASTERS', 'ATP MASTERS 1000 PARIS', 'PARIS'],
    "INTERNAZIONALI BNL D'ITALIA": [
        "INTERNAZIONALI BNL D'ITALIA",
        "ATP MASTERS 1000 ROME",
        "ROME",
    ],
    "WESTERN & SOUTHERN FINANCIAL GROUP MASTERS": ["WESTERN & SOUTHERN OPEN", "ATP MASTERS 1000 CINCINNATI", "CINCINNATI"],
    "AEGON CHAMPIONSHIPS": "LONDON / QUEEN'S CLUB",
    "MONTE CARLO MASTERS": "ATP MASTERS 1000 MONTE CARLO",
    "MUTUA MADRID OPEN": ["ATP MASTERS 1000 MADRID", "MUTUA MADRID OPEN"],
    "SHANGHAI MASTERS": ["ATP MASTERS 1000 SHANGHAI", "SHANGHAI"],
    "ROGERS MASTERS": ["ATP MASTERS 1000 CANADA", "COUPE ROGERS"],
    "CITI OPEN": ["WASHINGTON", "CITI OPEN"],
    "GERMAN TENNIS CHAMPIONSHIPS": [
        "GERMAN OPEN TENNIS CHAMPIONSHIPS",
        "HAMBURG",
        "ATP MASTERS 1000 HAMBURG",
        "HAMBURG EUROPEAN OPEN"
    ],
    "OPEN BANCO SABADELL": [
        "BARCELONA OPEN BANC SABADELL",
        "BARCELONA",
    ],
    "SWISS INDOORS": "BASEL",
    "RAKUTEN JAPAN OPEN TENNIS CHAMPIONSHIPS": ["RAKUTEN JAPAN OPEN TENNIS CHAMPIONSHIPS", "TOKYO"],
    "QATAR EXXON MOBIL OPEN": ["QATAR EXXONMOBIL OPEN", "DOHA"],
    "GRAND PRIX HASSAN II": "MARRAKECH",
    "GERRY WEBER OPEN": "HALLE",
    "CHINA OPEN": ["CHINA OPEN", "BEIJING"],
    "ABIERTO MEXICANO": ["ABIERTO MEXICANO TELCEL PRESENTADO POR HSBC", "ACAPULCO"],
    "DELRAY BEACH OPEN": ["DELRAY BEACH", "DELRAY BEACH OPEN BY VITACOST.COM"],
    "DUBAI TENNIS CHAMPIONSHIPS": ["DUBAI", "DUBAI DUTY FREE TENNIS CHAMPIONSHIPS"],
    "ERSTE BANK OPEN": ["VIENNA", "ERSTE BANK OPEN"],
    "ARGENTINA OPEN": ["BUENOS AIRES", "ARGENTINA OPEN"],
    "RICOH OPEN": ["S-HERTOGENBOSCH", "'S-HERTOGENBOSCH"],
    "EASTBOURNE INTERNATIONAL": ["EASTBOURNE", "NATURE VALLEY INTERNATIONAL"],
    "BB&T ATLANTA OPEN": ["BB&T ATLANTA OPEN", "ATLANTA"],
    "ASB CLASSIC": ["ASB CLASSIC", "AUCKLAND"],
    "SHENZHEN OPEN": ["SHENZHEN"],
    "SKISTAR SWEDISH OPEN": ["BASTAD", "SWEDISH OPEN"],
    "STOCKHOLM OPEN": ["STOCKHOLM", "INTRUM STOCKHOLM OPEN"],
    "ST. PETERSBURG OPEN": ["ST. PETERSBURG", "ST. PETERSBURG OPEN"],
    "SUISSE OPEN GSTAAD": ["GSTAAD", "J. SAFRA SARASIN SWISS OPEN GSTAAD"],
    "SYDNEY INTERNATIONAL": ["SYDNEY INTERNATIONAL", "SYDNEY"],
    "TATA OPEN": ["TATA OPEN MAHARASHTRA", "PUNE"],
    "U.S. MEN'S CLAY COURT CHAMPIONSHIPS": ["HOUSTON", "FAYEZ SAROFIM & CO. U.S. MEN'S CLAY COURT CHAMPIONSHIP"],
    "ANTALYA OPEN": ["ANTALYA", "TURKISH AIRLINES OPEN ANTALYA"],
    "CROATIA OPEN": ["PLAVA LAGUNA CROATIA OPEN UMAG", "UMAG"],
    "ECUADOR OPEN": "QUITO",
    "GAZPROM HUNGARIAN OPEN": ["HUNGARIAN OPEN", "BUDAPEST"],
    "GENERALI OPEN": ["GENERALI OPEN", "KITZBUHEL", "KITZBÜHEL", "KITZBUEHEL"],
    "OPEN DE MOSELLE": ["MOSELLE OPEN", "METZ"],
    "GENEVA OPEN": ["GENEVA", "BANQUE ERIC STURDZA GENEVA OPEN"],
    "GARANTI KOZA SOFIA OPEN": ["SOFIA", "SOFIA OPEN"],
    "EUROPEAN OPEN": ["ANTWERP", "EUROPEAN OPEN"],
    "HALL OF FAME CHAMPIONSHIPS": ["NEWPORT", "HALL OF FAME OPEN"],
    "ABIERTO MEXICANO MIFEL": ["LOS CABOS", "ABIERTO DE TENIS MIFEL PRESENTADO POR CINEMEX"],
    "ISTANBUL OPEN": "ISTANBUL",
    "KREMLIN CUP": ["MOSCOW", "VTB KREMLIN CUP"],
    "LYON OPEN": ["LYON", "OPEN PARC AUVERGNE-RHONE-ALPES LYON"],
    "MERCEDES CUP": ["STUTTGART", "ATP MASTERS 1000 STUTTGART", "MERCEDESCUP"],
    "MILLENNIUM ESTORIL OPEN": ["ESTORIL", "MILLENNIUM ESTORIL OPEN"],
    "BRASIL OPEN": ["SAO PAULO", "BRASIL OPEN"],
    "CHENGDU OPEN": ["CHENGDU", "CHENGDU OPEN"],
    "NEW YORK OPEN": ["NEW YORK", "NEW YORK OPEN"],
    "OPEN 13": ["OPEN 13 PROVENCE", "MARSEILLE"],
    "BMW OPEN": ["MUNICH", "BMW OPEN BY FWU"],
    "OPEN SUD DE FRANCE": ["OPEN SUD DE FRANCE", "MONTPELLIER"],
    "MASTERS CUP": ["TENNIS MASTERS CUP", "NITTO ATP FINALS"],
    "RIO OPEN": ["RIO DE JANEIRO", "RIO OPEN PRESENTED BY CLARO"],
    "ABN AMRO WORLD TENNIS TOURNAMENT": ["ROTTERDAM", "ABN AMRO WORLD TENNIS TOURNAMENT"],
    "WINSTON-SALEM OPEN AT WAKE FOREST UNIVERSITY": ['WINSTON-SALEM', 'WINSTON-SALEM OPEN']
}

#### Score Parsing

In [17]:
WALKOVER_DEFS = [
    'W/O',
    'DEF'
]

def parse_set_score(s):
    if 'RET' in s:
        return ('RETIRE', 'RETIRE')
    elif any([x in s for x in WALKOVER_DEFS]):
        return ('WALKOVER', 'WALKOVER')
    elif 'UNP' in s:
        return ('MATCH_NOT_PLAYED', 'MATCH_NOT_PLAYED')
    else:
        s = s.strip(' (NA)')
        if len(s) == 2:
            return int(s[0]), int(s[1])
        else:
            for b in range(1, len(s)):
                s1, s2 = int(s[:b]), int(s[b:])
                if abs(s1 - s2) <= 2:
                    return s1, s2
            return None, None 

In [18]:
def parse_match_score(s):
    if pd.isnull(s):
        return {}
    set_scores = s.split(';')
    out = {}
    for idx, ss in enumerate(set_scores):
        w, l = parse_set_score(ss)
        out.update({
            'W%d' % (idx + 1): w,
            'L%d' % (idx + 1): l
        })
    return out 

In [19]:
score_df = pd.DataFrame(jd['score'].map(parse_match_score).tolist())

In [20]:
score_df.head()

,L1,L2,L3,L4,L5,W1,W2,W3,W4,W5
0,3,2,6,1,NaN,6,6,5,6,NaN
1,3,3,5,NaN,NaN,6,6,6,NaN,NaN
2,5,6,4,1,NaN,6,5,6,6,NaN
3,2,5,2,NaN,NaN,6,6,6,NaN,NaN
4,1,2,4,NaN,NaN,6,6,6,NaN,NaN


In [21]:
jd = pd.concat([jd, score_df], axis=1)
jd.shape

(373236, 40)

In [22]:
score_cols = ['W%d' % s for s in range(1, 6)] + ['L%d' % s for s in range(1, 6)]

#### Joining

In [173]:
odds_df['odds_match_id'] = range(odds_df.shape[0])

In [174]:
jd_2018 = jd[jd['year'] == 2018].copy()

In [175]:
join_cols = sorted([
    'winner_last_name',
    'loser_last_name',
    'tourney_title'
] + score_cols)

In [176]:
def get_tourney(x, idx):
    mapped = TOURNAMENT_MAPPING[x]
    if isinstance(mapped, str):
        return mapped if idx == 0 else None
    if idx >= len(mapped):
        return None
    else:
        return mapped[idx]

idx = 0
merged_dfs = []
while(True):
    print(idx)
    cur_odds = odds_df.copy()
    cur_odds['tourney_title'] = odds_df['Tournament'].map(lambda x: get_tourney(x, idx))
    if cur_odds['tourney_title'].isnull().all():
         break
    merged_dfs.append(
        pd.merge(
            jd_2018,
            cur_odds,
            on=join_cols
        )
    )
    idx += 1

0
1
2
3
4


In [177]:
all_merged = pd.concat(merged_dfs)

In [202]:
all_merged.to_csv("./merged_2018.csv", index=False)

In [178]:
assert all_merged['match_id'].value_counts().max() == 1

In [179]:
missing = odds_df[~odds_df['odds_match_id'].isin(all_merged['odds_match_id'].tolist())]

In [180]:
missing.shape

(82, 43)

In [181]:
missing['Comment'].value_counts()

Retired     68
Walkover    14
Name: Comment, dtype: int64

In [182]:
missing[missing['Comment'] == 'Completed'][[
    'Winner', 'Loser', 'winner_last_name', 'loser_last_name'
]]

,Winner,Loser,winner_last_name,loser_last_name


In [183]:
# What tournaments are ALWAYS missing!?
missing[~missing['Tournament'].isin(all_merged['Tournament'])]['Tournament'].value_counts()

Series([], Name: Tournament, dtype: int64)

In [184]:
missing[missing['Tournament'] == "CHENGDU OPEN"].iloc[0]

ATP                           56
Location                 Chengdu
Tournament          CHENGDU OPEN
Date                     9/25/18
Series                    ATP250
Court                    Outdoor
Surface                     Hard
Round                  1st Round
Best of                        3
Winner              BEMELMANS R.
Loser                  ZVEREV M.
WRank                        114
LRank                         69
WPts                         492
LPts                         790
W1                             6
L1                             3
W2                             2
L2                             0
W3                           NaN
L3                           NaN
W4                           NaN
L4                           NaN
W5                           NaN
L5                           NaN
Wsets                          1
Lsets                          0
Comment                  Retired
B365W                        1.9
B365L                        1.8
EXW       

In [185]:
TOURNAMENT_MAPPING['FRENCH OPEN']

'ROLAND GARROS'

In [186]:
get_tourney('FRENCH OPEN', 0)

'ROLAND GARROS'

In [187]:
def inspect_match(w=None, l=None):
    if w is not None:
        rel = jd_2018[
            (jd_2018['winner_last_name'] == w)
        ]
    else:
        rel = jd_2018
    if l is not None:
        rel = rel[
            rel['loser_last_name'] == w
        ]
    return rel[[
        'winner_last_name',
        'loser_last_name',
        'tourney_title'
    ] + score_cols]

inspect_match('POLANSKY')

,winner_last_name,loser_last_name,tourney_title,W1,W2,W3,W4,W5,L1,L2,L3,L4,L5
350999,POLANSKY,PELIWO,BRISBANE,6,6,NaN,NaN,NaN,2,2,NaN,NaN,NaN
351003,POLANSKY,HANFMANN,BRISBANE,6,3,6,NaN,NaN,1,6,3,NaN,NaN
351474,POLANSKY,DOMINGUES,AUSTRALIAN OPEN,7,6,NaN,NaN,NaN,6,4,NaN,NaN,NaN
351528,POLANSKY,HARRISON,AUSTRALIAN OPEN,6,7,NaN,NaN,NaN,2,5,NaN,NaN,NaN
352592,POLANSKY,MENENDEZ-MACEIRAS,DELRAY BEACH,6,6,NaN,NaN,NaN,2,3,NaN,NaN,NaN
352984,POLANSKY,KING,INDIAN WELLS,6,6,NaN,NaN,NaN,4,2,NaN,NaN,NaN
353048,POLANSKY,KOZLOV,INDIAN WELLS,6,6,NaN,NaN,NaN,2,4,NaN,NaN,NaN
353242,POLANSKY,AREVALO,ATP MASTERS 1000 INDIAN WELLS,7,6,NaN,NaN,NaN,5,3,NaN,NaN,NaN
353253,POLANSKY,BERRETTINI,ATP MASTERS 1000 INDIAN WELLS,6,1,6,NaN,NaN,3,6,3,NaN,NaN
353282,POLANSKY,COPIL,ATP MASTERS 1000 INDIAN WELLS,7,6,7,NaN,NaN,6,7,6,NaN,NaN


In [194]:
ranked = all_merged[
    all_merged['WRank'].notnull() &
    all_merged['LRank'].notnull()
]

In [195]:
(ranked['WRank'] < ranked['LRank']).mean()

0.6354737666405639

In [199]:
(ranked['B365W'] <= ranked['B365L']).mean()

0.692247454972592

In [200]:
(ranked['B365W'] < ranked['B365L']).mean()

0.6644479248238058

In [208]:
import numpy as np

with_scores = jd[jd['score'].notnull()]
winner_sets = np.zeros(with_scores.shape[0])
for set_index in range(1, 6):
    winner_sets += (with_scores['W%d' % set_index] > with_scores['L%d' % set_index]).astype(int)

In [211]:
pd.Series(winner_sets).value_counts()

2.0    320016
3.0     41503
0.0      5891
1.0      5668
4.0         7
dtype: int64

In [217]:
with_scores[winner_sets == 1][['score', 'tourney_url_suffix']]

,score,tourney_url_suffix
188,46;46;86;20 (RET),/en/scores/archive/wimbledon/540/1902/results
257,62;53 (RET),/en/scores/archive/australasian-championships/...
344,64;46;26;62 (RET),/en/scores/archive/wimbledon/540/1911/results
657,64;64 (RET),/en/scores/archive/us-championships/560/1914/r...
681,61;46;61 (RET),/en/scores/archive/us-championships/560/1914/r...
777,62;62 (RET),/en/scores/archive/australasian-championships/...
793,75;62 (RET),/en/scores/archive/australasian-championships/...
1053,63;26;11 (RET),/en/scores/archive/us-championships/560/1916/r...
1305,46;26;60;35 (RET),/en/scores/archive/us-championships/560/1919/r...
1367,63;61 (RET),/en/scores/archive/australasian-championships/...


In [221]:
with_scores[with_scores['score'].map(lambda x: 'W/O' in x)][['winner_name', 'loser_name']]

,winner_name,loser_name
10,JOHN HARTLEY,FRANK HADOW
28,CLARENCE CLARK,JAMES DWIGHT
66,ERNEST RENSHAW,C. LACYSWEET
68,HERBERT LAWFORD,ERNEST RENSHAW
69,HERBERT LAWFORD,WILLIAM RENSHAW
81,HENRY SLOCUM,RICHARD SEARS
101,WILFRED BADDELEY,WILLOUGHBY HAMILTON
121,ROBERT WRENN,OLIVER CAMPBELL
122,WILFRED BADDELEY,T. CHAYTOR
130,WILFRED BADDELEY,HERBERT BADDELEY


In [ ]:
with_scores